<a href="https://www.kaggle.com/code/dheerajkr1a1a/chennai-retail-prices-forecast?scriptVersionId=152664267" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Why only Chennai?
It is the city with largest datasize and maximum arrival quantity across all centres.

Source:
https://www.kaggle.com/code/dheerajkr1a1a/tomato-centre-wise-data-analysis

# Objective
The goal is to utilize the prophet multivariate time series model to forecast the spot prices of tomatoes at the Chennai center, incorporating both macroeconomic and climate-related variables.

# Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cufflinks as cf
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default='plotly_white'
import os
# import plotly.io as pio
# pio.renderers.default = 'colab'
import seaborn as sns
import plotly.express as px
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation
#from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots
import itertools
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

# Reading & plotting complete data for Ranchi center

In [ ]:
parent_df={}
parent_df['RANCHI']=pd.read_csv('/kaggle/input/chennai-tomato-retail-prices-analysis/complete_data.csv',index_col=0,parse_dates=[0])

In [ ]:
# sns.lineplot(data=parent_df['RANCHI'])

parent_df['RANCHI'].iplot()

# Plotting only Retail Prices & Arrival Quantity

In [ ]:
px.line(parent_df['RANCHI'],x=parent_df['RANCHI'].index,y=['Retail Prices','Arrival Quantity'])

# Plotting Retail Prices

In [ ]:
px.line(parent_df['RANCHI'],x=parent_df['RANCHI'].index,y='Retail Prices')

# Plotting Arrival Quantity

In [ ]:
parent_df['RANCHI']['Arrival Quantity'].iplot(xTitle='Date',yTitle='Arrival Quantity')

# Dropping the outliers with retail prices less than threshold set for keeping the data
**For example here 10k is the threshold and three values are 9k,8k and 4.5k being the local outliers will not be dropped.**

In [ ]:
parent_df['RANCHI'].drop(['2014-01-30','2014-02-10','2011-12-07'],inplace=True)

# Looking for the changes

In [ ]:
px.line(parent_df['RANCHI'],x=parent_df['RANCHI'].index,y=['Retail Prices','Arrival Quantity'])

In [ ]:
parent_df['RANCHI'].loc['2010':'2019','Arrival Quantity'].isnull().sum()

# Filling null with -1 to prevent them from being lost when applying condition of dropping outliers

In [ ]:
parent_df['RANCHI']['Arrival Quantity'].fillna(-1,inplace=True)

In [ ]:
parent_df['RANCHI'].info()

# Condition of dropping outliers

In [ ]:
parent_df['RANCHI']=parent_df['RANCHI'][(parent_df['RANCHI']['Arrival Quantity']<=500)]

In [ ]:
parent_df['RANCHI'].info()

# Arrival qty without outliers

In [ ]:
px.line(parent_df['RANCHI'],x=parent_df['RANCHI'].index,y=['Arrival Quantity'])

# Getting back the null values for easier analysis

In [ ]:
parent_df['RANCHI']['Arrival Quantity'].replace(-1,np.nan,inplace=True)

In [ ]:
parent_df['RANCHI'].info()

In [ ]:
px.line(parent_df['RANCHI'],x=parent_df['RANCHI'].index,y=['Arrival Quantity'])

# Filling null with 1000 to prevent them from being lost when applying condition of dropping downward outliers

In [ ]:
parent_df['RANCHI']['Arrival Quantity'].fillna(1000,inplace=True)

In [ ]:
parent_df['RANCHI'].info()

# Condition of dropping downward outliers

In [ ]:
parent_df['RANCHI']=parent_df['RANCHI'][(parent_df['RANCHI']['Arrival Quantity']>=180)]

In [ ]:
parent_df['RANCHI'].info()

# Getting back the null values for easier analysis

In [ ]:
parent_df['RANCHI']['Arrival Quantity'].replace(1000,np.nan,inplace=True)

In [ ]:
parent_df['RANCHI'].info()

In [ ]:
px.line(parent_df['RANCHI'],x=parent_df['RANCHI'].index,y=['Arrival Quantity'])

In [ ]:
df_Ranchi=parent_df['RANCHI'].copy()

In [ ]:
df_Ranchi['2010':'2020'].describe()

# Visualising the data with outliers 

In [ ]:
df_Ranchi['2010':'2020']['Retail Prices'].iplot()

# Creating cleaned copy of data for ACF plots and adf test 

In [ ]:
df_Ranchi_1=df_Ranchi.copy()
df_Ranchi.isnull().sum()

# Dropping Retail Prices Outliers

In [ ]:
# https://www.ceicdata.com/en/india/retail-price-department-of-agriculture-and-cooperation-food-by-cities-tomato/retail-price-doac-tomato-tamil-nadu-chennai

df_Ranchi_1=df_Ranchi_1[df_Ranchi_1['Retail Prices']<10000]
df_Ranchi_1.isnull().sum()


In [ ]:
df_Ranchi=df_Ranchi['2010':'2020'].copy()


In [ ]:
df_Ranchi_1=df_Ranchi_1['2010':'2020'].copy()
df_Ranchi_1.isnull().sum()

In [ ]:
df_Ranchi_dropped=df_Ranchi_1.dropna()

In [ ]:
df_Ranchi_dropped['Arrival Quantity'].iplot()

In [ ]:
px.line(df_Ranchi_dropped,x=df_Ranchi_dropped.index,y='Retail Prices')

In [ ]:
df_Ranchi_dropped.loc[:,['Arrival Quantity','Retail Prices']].iplot(subplots=True)

In [ ]:
pd.plotting.autocorrelation_plot(df_Ranchi_dropped['2016':'2020']['Arrival Quantity'])

In [ ]:
pd.plotting.autocorrelation_plot(df_Ranchi_dropped['2010':'2020']['Arrival Quantity'].resample('1m').max())

In [ ]:
pd.plotting.autocorrelation_plot(df_Ranchi_dropped['2016':'2020']['Retail Prices'])

In [ ]:
pd.plotting.autocorrelation_plot(df_Ranchi_dropped['2016':'2020']['Retail Prices'].resample('1m').max())

In [ ]:
pd.plotting.lag_plot(df_Ranchi_1['Arrival Quantity'],lag=1)

In [ ]:
pd.plotting.lag_plot(df_Ranchi['Retail Prices'],lag=1)

In [ ]:
from statsmodels.tsa.stattools import adfuller


# In[19]:


test_result=adfuller(df_Ranchi_dropped['Arrival Quantity'])


# In[20]:


#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(sales):
    result=adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")


In [ ]:
adfuller_test(df_Ranchi_dropped['Arrival Quantity'])

In [ ]:
adfuller_test(df_Ranchi_dropped['Retail Prices'])

In [ ]:
df_Ranchi.isnull().sum()

# Applying similar procedure to drop outliers without losing NULL

In [ ]:
df_Ranchi.fillna(-1,inplace=True)
df_Ranchi=df_Ranchi[df_Ranchi['Retail Prices']<10000]
df_Ranchi.replace(-1,np.nan,inplace=True)

In [ ]:
df_Ranchi

# Imputing with rolling window size = adf lags

In [ ]:
df_Ranchi_imp=pd.DataFrame()
df_Ranchi_imp['Retail Prices']=df_Ranchi['Retail Prices'].rolling(window=4,min_periods=2).mean().fillna(method='ffill')
df_Ranchi_imp['Arrival Quantity']=df_Ranchi['Arrival Quantity'].rolling(window=22,min_periods=11).mean().fillna(method='ffill')
df_Ranchi_imp['Retail Prices_lag']=df_Ranchi['Retail Prices'].rolling(window=4,min_periods=2).mean().fillna(method='ffill').shift(1)
# df_Ranchi_imp['Arrival Quantity']=df_Ranchi['Arrival Quantity'].rolling(window=5,min_periods=2).mean().fillna(method='bfill')
df_Ranchi_imp=df_Ranchi_imp.dropna()
#df_Ranchi_imp['2008':'2016']['Retail Prices'].plot()
df_Ranchi_imp[['Retail Prices','Retail Prices_lag']].iplot()
df_Ranchi_imp.isnull().sum()

# Resampling daily data into monthly data

In [ ]:
df_Ranchi_imp_month=pd.DataFrame()
df_Ranchi_imp_month['Retail Prices_lag']=df_Ranchi_imp['Retail Prices_lag'].resample('M').mean()
df_Ranchi_imp_month['Arrival Quantity']=df_Ranchi_imp['Arrival Quantity'].resample('M').max()
df_Ranchi_imp_month['Arrival Quantity_mean']=df_Ranchi_imp['Arrival Quantity'].resample('M').sum()
df_Ranchi_imp_month=df_Ranchi_imp_month.dropna()
df_Ranchi_imp_month.index = df_Ranchi_imp_month.index + pd.offsets.MonthBegin(0)
df_Ranchi_imp_month[['Arrival Quantity','Retail Prices_lag']].tail(6)

In [ ]:
df_Ranchi_imp_month

# Reading csv file (Fertilizer subsidy rates, Diesel Reserves and Diesel Prices) 

In [ ]:
df_1_new=pd.read_csv('/kaggle/input/chennai-tomato-retail-prices-analysis/Fert_chennai_dies_price.csv',index_col=[0],parse_dates=True)
df_1_new

In [ ]:
#df_1_new=pd.read_csv('FERTILIZERS.csv',names=['Date','N','P','K','S'],header=0,index_col=[0],parse_dates=True)
finaldf1=df_Ranchi_imp_month[['Arrival Quantity_mean','Retail Prices_lag']].copy()
finaldf1

# Merging original df with extra features

In [ ]:
finaldf1=df_Ranchi_imp_month[['Arrival Quantity_mean','Retail Prices_lag']].copy()
df_concat_1 = pd.concat([finaldf1, df_1_new], axis=1, join='outer')
df_concat_1.loc[:,['N','P','K','S','thousand million barrels','dprice']]=df_concat_1.loc[:,['N','P','K','S','thousand million barrels','dprice']].interpolate().fillna(method='bfill')
df_concat_1.dropna(subset=['Arrival Quantity_mean','Retail Prices_lag'],inplace=True)
df_concat_1.head()

# Creating meaningful features like NBS rates using N,P,K 

In [ ]:
df_concat_1['NBS rate']=3*df_concat_1['N']+1.5*df_concat_1['P']+df_concat_1['K']
df_concat_1=df_concat_1.loc[:,['Arrival Quantity_mean','Retail Prices_lag','NBS rate','thousand million barrels','dprice']]
df_concat_1

# Reading Rainfall data, resampling it using sum attribute and cropping according to original indexes

In [ ]:
df_rain=pd.read_csv('/kaggle/input/chennai-tomato-retail-prices-analysis/Rainfall_ponneri.csv',index_col=[0],parse_dates=True)
df_rain

In [ ]:
df_rain_m=df_rain.resample('MS').sum()
df_rain_m

In [ ]:
df_rain_m=df_rain_m.loc[df_concat_1.index,:]
df_rain_m

# Reading Temp data, and cropping according to original indexes

In [ ]:
df_temp=pd.read_csv('/kaggle/input/chennai-tomato-retail-prices-analysis/Ponneri_temp_data.csv',index_col=[0],parse_dates=True)
df_temp

In [ ]:
df_temp=df_temp.loc[df_concat_1.index[:-4],'tas']
df_temp

In [ ]:
df_concat_1=pd.concat([df_concat_1,df_rain_m,df_temp],axis=1)
df_concat_1

# Merging GDP and CPI onto new dataframe

In [ ]:
df_new=pd.read_csv('/kaggle/input/chennai-tomato-retail-prices-analysis/gdp and cpi merged.csv',index_col=0,parse_dates=True)
finaldf1=df_concat_1.merge(df_new,on=df_concat_1.index)
finaldf1=finaldf1.iloc[:-4,:]

In [ ]:
finaldf1

# Making it ready-to-use for prophet model 

In [ ]:
finaldf1['ds'] = pd.to_datetime(finaldf1.key_0)
finaldf1.drop('key_0',axis=1,inplace=True)
finaldf1
finaldf1.rename(columns={'Retail Prices_lag':'y'},inplace=True)
finaldf1.tail()

In [ ]:
finaldf1.iplot()

# Repeating the same process of concatenation for max (arrival qty)

In [ ]:
df_1_new

In [ ]:
df_Ranchi_imp_month[['Arrival Quantity','Retail Prices_lag']]

In [ ]:
finaldf=df_Ranchi_imp_month[['Arrival Quantity','Retail Prices_lag']].copy()
df_concat = pd.concat([finaldf, df_1_new], axis=1, join='outer')
df_concat.loc[:,['N','P','K','S','thousand million barrels','dprice']]=df_concat.loc[:,['N','P','K','S','thousand million barrels','dprice']].interpolate().fillna(method='bfill')
df_concat.dropna(subset=['Arrival Quantity','Retail Prices_lag'],inplace=True)
df_concat.tail()

In [ ]:
df_concat['NBS rate']=3*df_concat['N']+1.5*df_concat['P']+df_concat['K']
df_concat=df_concat.loc[:,['Arrival Quantity','Retail Prices_lag','NBS rate','thousand million barrels','dprice']]
df_concat

In [ ]:
df_concat=pd.concat([df_concat,df_rain_m,df_temp],axis=1)
df_concat

In [ ]:
df_new=pd.read_csv('/kaggle/input/chennai-tomato-retail-prices-analysis/gdp and cpi merged.csv',index_col=0,parse_dates=True)
finaldf=df_concat.merge(df_new,on=df_concat.index)
finaldf=finaldf.iloc[:-4,:]
finaldf

In [ ]:
finaldf['ds'] = finaldf['key_0']
finaldf.drop('key_0',axis=1,inplace=True)
finaldf
finaldf.rename(columns={'Retail Prices_lag':'y'},inplace=True)
finaldf.tail()

In [ ]:
finaldf.iplot(subplots=True)

In [ ]:
finaldf['ds']

In [ ]:
finaldf.isnull().sum()

# Checking correlations

In [ ]:
sns.heatmap(data=finaldf1.corr())

In [ ]:
sns.heatmap(data=finaldf.corr())

# Dropping Diesel reserves,CPI and NBS rate

In [ ]:
finaldf.drop(['thousand million barrels','CPI','NBS rate'],axis=1,inplace=True)
finaldf.columns

In [ ]:
finaldf1.drop(['thousand million barrels','CPI','NBS rate'],axis=1,inplace=True)
finaldf1.columns

# Using simple prophet model

In [ ]:
# splitting train and test data
trainDataSize = 0.80
splitSize = int(finaldf.shape[0]*trainDataSize)
print(splitSize)
train = finaldf[0:splitSize]
test = finaldf[splitSize:]
print(train.shape)
print(test.shape)
Columns=finaldf.columns.tolist()
Columns.remove('y')
Columns.remove('ds')
prophetColumns =Columns
# model

model = Prophet()

# adding all columns in add regressor
for col in prophetColumns:
    model.add_regressor(col)
#training model
model.fit(train)

future =test.copy()
future.drop('y',axis=1,inplace=True)

#prediction
prediction = model.predict(future)

In [ ]:
y_true = test['y'].values
y_pred = prediction['yhat'].values


plt.figure(figsize=(12, 6))
mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)
mape = mean_absolute_percentage_error(y_true, y_pred)
print("MAPE:", mape)
# plot expected vs actual
plt.plot(y_true, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=test['ds'].values, y=y_true, name="actual targets"),)
    #secondary_y=False,)
fig.add_trace(
    go.Scatter(x=test['ds'].values, y=y_pred, name="predicted targets"),)
    #secondary_y=True,)
fig.add_trace(go.Scatter(x=finaldf['ds'].values, y=finaldf['y'].values, name="original data"),)
    #secondary_y=True,)
fig.update_layout(
    title_text="Actual vs Predicted Targets")
fig.update_xaxes(title_text="Timeline")
fig.update_yaxes(title_text="actual targets", secondary_y=False)
fig.update_yaxes(title_text="predicted targets", secondary_y=True)
fig.show()

# Modifying trend flexibility & amount of historical data to be included for detecting trend changepoints.

In [ ]:
#Trial 2

# splitting train and test data
trainDataSize = 0.80
splitSize = int(finaldf.shape[0]*trainDataSize)
print(splitSize)
train = finaldf[0:splitSize]
test = finaldf[splitSize:]
print(train.shape)
print(test.shape)
Columns=finaldf.columns.tolist()
Columns.remove('y')
Columns.remove('ds')
prophetColumns =Columns
# model

#seasonality_mode = 'multiplicative'

#modelWeekly = Prophet(changepoint_range=0.90,changepoint_prior_scale = 0.5)

model = Prophet(changepoint_range=0.95,changepoint_prior_scale = 1,)#seasonality_prior_scale=20)

# adding all columns in add regressor
for col in prophetColumns:
    model.add_regressor(col)
#training model
model.fit(train)

future =test.copy()
future.drop('y',axis=1,inplace=True)

#prediction
prediction = model.predict(future)

y_true = test['y'].values
y_pred = prediction['yhat'].values


plt.figure(figsize=(10, 5))
mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)
mape = mean_absolute_percentage_error(y_true, y_pred)
print("MAPE:", mape)
# plot expected vs actual
plt.plot(y_true, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

In [ ]:
# with extreme weather and 80% train data, seasonality mode mulitplicative
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=test['ds'].values, y=y_true, name="actual targets"),)
    #secondary_y=False,)
fig.add_trace(
    go.Scatter(x=test['ds'].values, y=y_pred, name="predicted targets"),)
    #secondary_y=True,)
fig.add_trace(go.Scatter(x=finaldf['ds'].values, y=finaldf['y'].values, name="original data"),)
    #secondary_y=True,)
fig.update_layout(
    title_text="Actual vs Predicted Targets")
fig.update_xaxes(title_text="Timeline")
fig.update_yaxes(title_text="actual targets", secondary_y=False)
fig.update_yaxes(title_text="predicted targets", secondary_y=True)
fig.show()

In [ ]:
finaldf.info()

# Implementing prophet model with 95% cofidence interval

In [ ]:
def model_prophet(Prophetdf,n_obs= 113):

    Prophetdf_train=Prophetdf[0:n_obs]
    Prophetdf_test =Prophetdf[n_obs:]
    multi_model = Prophet(interval_width = 0.95)
    Prophet_cols = Prophetdf.columns.tolist()
    Prophet_cols.remove('ds')
    Prophet_cols.remove('y')
    print(Prophetdf_train.columns)
    #print(Prophetdf_test[['ds','y']])

    # adding all columns in add regressor
    for col in Prophet_cols:
        multi_model.add_regressor(col)


    multi_model.fit(Prophetdf_train)
    print('Prophetdf_test.shape[0]')
    print(Prophetdf_test.shape[0])
    # make furture frame
    future = multi_model.make_future_dataframe(periods = Prophetdf_test.shape[0],freq='MS', include_history=True)
    for col in Prophet_cols:
        future[col] =Prophetdf[col]
    print(Prophetdf_test[['ds','y']])
    print('future')
    print(future)

    # predict future
    forecastProphet =  multi_model.predict(future)
    combined_df = pd.merge(Prophetdf_test[['ds','y']],forecastProphet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds')

    print(forecastProphet[['ds','yhat', 'yhat_lower','yhat_upper']].tail())

    fig1 = multi_model.plot(forecastProphet)

    Prophetdf.plot(x = 'ds', y = 'y')

    multi_model.plot_components(forecastProphet);


    #print(Prophetdf_test[['ds','y']])
    print(forecastProphet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']][-6:])

    combined_df = pd.merge(Prophetdf_test[['ds','y']],forecastProphet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']][-6:], on='ds')
    #print(combined_df.head())


    #Check MAE value
    MAE = mean_absolute_error(combined_df['y'], combined_df['yhat'])
    print('MAE')
    print(MAE)

#Check MAPE value
    MAPE = mean_absolute_percentage_error(combined_df['y'], combined_df['yhat'])
    print('MAPE')
    print(MAPE)

    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.rcParams.update({'figure.figsize':  (15, 3),  'figure.dpi' : 300})
    fig, ax = plt.subplots()

    sns.lineplot(data = Prophetdf , x = 'ds', y = 'y' ,label = 'Original')
    sns.lineplot(data= forecastProphet, x = 'ds', y = 'yhat', label = 'Forecast')
    plt.grid(linestyle = '-', linewidth = 0.3)

    plt.legend(["Original","forecast"])


    import seaborn as sns
    import matplotlib.pyplot as plt

    plt.rcParams.update({'figure.figsize':  (15, 3),  'figure.dpi' : 300})
    fig, ax = plt.subplots()
    sns.lineplot(data = Prophetdf[-6:] , x = 'ds', y = 'y' ,label = 'Original')
    sns.lineplot(data= forecastProphet[-6:], x = 'ds', y = 'yhat', label = 'Forecast')
    plt.grid(linestyle = '-', linewidth = 0.3)
    plt.legend(["Original","forecast"])




    return MAE

In [ ]:
finaldf.head()

In [ ]:
finaldf1.head()

In [ ]:
finaldf1.iplot(subplots=True)

In [ ]:
#HERE we can see that MAE for arrival quantity mean(589.63) resampling is significantly less than that calculated using max resampling(608.94)

# For arrival (max attribute) 

In [ ]:
MAE_Prophet = model_prophet(finaldf)

# For arrival (mean attribute) 

In [ ]:
MAE_Prophet_1=model_prophet(finaldf1)